In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    default_data_collator
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# Accelerator 패치: __init__ 메서드를 안전하게 한 번만 패치 (커널 재시작 후 실행 권장)
from accelerate import Accelerator
if not hasattr(Accelerator, "_original_init"):
    Accelerator._original_init = Accelerator.__init__
    def patched_accelerator_init(self, *args, **kwargs):
        kwargs["device_placement"] = False
        Accelerator._original_init(self, *args, **kwargs)
    Accelerator.__init__ = patched_accelerator_init

/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig

BASE_MODEL = "google/gemma-2b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 베이스 모델 로드 (4-bit)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map={"": 0}  # 예: 단일 GPU 사용
)
base_model = prepare_model_for_kbit_training(base_model)

# 저장한 LoRA 어댑터 적용
model = PeftModel.from_pretrained(base_model, "path/to/save/lora_adapters_ver1.0/")

# 이후 추가 튜닝을 위한 Trainer 구성 후 학습 진행
# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.69it/s]


In [5]:
# CSV 파일 경로
data_file = 'extracted_documents_사설.csv'

# CSV 파일로부터 데이터셋 로드
dataset = load_dataset("csv", data_files={"train": data_file})
print("데이터셋 로드 완료.")
dataset

데이터셋 로드 완료.


DatasetDict({
    train: Dataset({
        features: ['original_text', 'summary_text'],
        num_rows: 56760
    })
})

In [6]:
def is_not_empty(example):
    # None일 경우 빈 문자열("")으로 대체 후 strip() 적용
    orig = (example.get("original_text") or "").strip()
    summ = (example.get("summary_text") or "").strip()
    return orig != "" and summ != ""

# 원본 train 데이터셋의 총 행 개수
original_count = dataset["train"].num_rows

# 빈 문자열 혹은 None이 포함된 행을 제거한 새로운 데이터셋 생성
#filtered_train = dataset["train"].filter(is_not_empty)
dataset["train"] = dataset["train"].filter(is_not_empty)

# 필터링 후 행 개수
filtered_count = dataset["train"].num_rows

# 제거된 행의 수 계산
removed_count = original_count - filtered_count

print(f"빈문자열(또는 None)이 있는 행의 개수: {removed_count}")
print(f"필터링 후 train 데이터셋 행 개수: {filtered_count}")


빈문자열(또는 None)이 있는 행의 개수: 4
필터링 후 train 데이터셋 행 개수: 56756


In [7]:
def preprocess_function(example):
    # 프롬프트 생성: 원문과 요약 요청을 명시
    prompt = "원문:\n" + example["original_text"] + "\n\n요약:\n"
    target = example["summary_text"]
    full_text = prompt + target

    # 전체 텍스트 토큰화 (truncation, padding 적용)
    tokenized_full = tokenizer(full_text, truncation=True) #, padding="max_length", max_length=512)
    
    # 프롬프트 토큰 길이 계산 (패딩 전 실제 길이)
    prompt_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
    prompt_length = len(prompt_ids)
    
    # 전체 토큰 시퀀스에서 프롬프트 부분은 손실 계산에서 제외 (-100 처리)
    labels = tokenized_full["input_ids"].copy()
    for i in range(prompt_length):
        if i < len(labels):
            labels[i] = -100
    tokenized_full["labels"] = labels
    return tokenized_full

In [8]:
# 5. 데이터셋에 전처리 함수 적용 (각 예제를 개별적으로 처리)
tokenized_dataset = dataset["train"].map(preprocess_function, batched=False)#, device_map={"": 1})
# 학습에 필요한 "input_ids", "attention_mask", "labels"만 남김
cols_to_remove = [col for col in tokenized_dataset.column_names if col not in ["input_ids", "attention_mask", "labels"]]
tokenized_dataset = tokenized_dataset.remove_columns(cols_to_remove)
print("토큰화 및 전처리 완료.")

tokenized_dataset


토큰화 및 전처리 완료.


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 56756
})

In [13]:
tokenized_dataset


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 56756
})

In [10]:
tokenized_dataset.to_csv("tokenized_dataset_사설.csv", index=False, encoding="utf-8-sig")

Creating CSV from Arrow format: 100%|██████████| 57/57 [00:51<00:00,  1.11ba/s]


552243167

In [16]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./qlora_gemma",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    learning_rate=5e-5,
    fp16=True,
    logging_steps=2000,
    save_steps=1000,
    save_total_limit=2,
    #push_to_hub=False,
    report_to="none",
    optim="adamw_torch", #"adamw_torch",
    max_grad_norm=1.0,
)

# Trainer 초기화
trainer = Trainer(
    model=model,#.to(device),
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=default_data_collator
)
print("Trainer 초기화 완료.")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Trainer 초기화 완료.


In [17]:
# 학습 시작
trainer.train()
print("학습 완료.")


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


AssertionError: No inf checks were recorded for this optimizer.